In [1]:
from os import fsencode, listdir, fsdecode
import imutils
import numpy as np
import cv2

directory = fsencode('images')
for file in listdir(directory)[:1]:
    filename = fsdecode(file)
    #img = Image.open('images/' + filename)
    
    # largest_area=0
    image = cv2.imread('images/' + filename)
    ratio = image.shape[0] / 300.0
    orig = image.copy()
    image = imutils.resize(image, height = 300)
    cv2.imshow('Resized', image)
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    edged = cv2.Canny(gray, 30, 200)
    
    (returned_image, cnts, hierarchy) = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:10]
    screenCnt = None
    for c in cnts:
        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)
     
        # if our approximated contour has four points, then
        # we can assume that we have found our screen
        if len(approx) == 4:
            screenCnt = approx
            break
    #print(screenCnt)        
    cv2.drawContours(image, [cnts[0].astype(int)], -1, (0, 255, 0), 3)
    cv2.imshow("Game Boy Screen", image)
    #cv2.imshow('edged ',edged)
    # img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # ret, thresh = cv2.threshold(img_gray, 40, 255, 0)
    # image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    # bounding_rect = None
    # for cnt in contours:
    #     area = cv2.contourArea(cnt)
    #     if (area>largest_area):
    #         largest_area=area
    #         bounding_rect=cv2.boundingRect(cnt)
    # rect=img(bounding_rect).clone()
    # cv2.imshow('largest contour ',rect)
    
    # # Sample 10 pixels from where there should always be solid background
    # samples = 0
    # for i in range(1000, 1010):
    #     samples += sum(img.getpixel((i, 100))) / 3
    # sample_average = samples / 10
    # # print('Sample average is {}'.format(sample_average))
    # center_x = int(img.width / 2)
    # center_y = int(img.height / 2)
    # upper_edge_y = None
    # for i in range(0, img.height):
    #     average_pixel_rgb = sum(img.getpixel((center_x, i))) / 3
    #     # print (average_pixel_rgb)
    #     if not (sample_average * 0.95 < average_pixel_rgb < sample_average * 1.05):
    #         # We have likely hit the upper edge of the real image
    #         draw = ImageDraw.Draw(img)
    #         draw.line((0, i, img.width, i), fill=128)
    #         # img.show()
    #         upper_edge_y = i
    #         break
    # 
    # lower_edge_y = None
    # for i in range(img.height - 1, 0, -1):
    #     average_pixel_rgb = sum(img.getpixel((center_x, i))) / 3
    #     # print (average_pixel_rgb)
    #     if not (155 < average_pixel_rgb < 170):
    #         # We have likely hit the lower edge of the real image
    #         draw = ImageDraw.Draw(img)
    #         draw.line((0, i, img.width, i), fill=128)
    #         # img.show()
    #         lower_edge_y = i
    #         break
    #         
    # right_edge_x = None
    # for i in range(img.width - 1, 0, -1):
    #     average_pixel_rgb = sum(img.getpixel((i, center_y))) / 3
    #     # print (average_pixel_rgb)
    #     if not (155 < average_pixel_rgb < 170):
    #         # We have likely hit the left edge of the real image
    #         draw = ImageDraw.Draw(img)
    #         draw.line((i, 0, i, img.height), fill=128)
    #         # img.show()
    #         right_edge_x = i
    #         break
    # 
    # img.show()
            
    # TODO: Find where the sciency colorgram-thing ends and left gray area begins, from there do regular detection

